In [ ]:
import pandas as pd

import nltk
import spacy

import string
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline 
from sklearn.metrics import classification_report

In [1]:


df2021= pd.read_csv('data/zeus_download_doc_digitalizzati20210514.csv', sep="|", encoding='latin')
df2021 = df2021.applymap(lambda x: x.lower() if isinstance(x, str) else x)
df2021.loc[df2021['famiglia'] == 'https://www.epc.bnl.it/bnl/filedownload.ashx?i=1&codicefase=5246330&num_prat=559377', 'famiglia'] = 'altro'
df2021.loc[df2021['famiglia'] == 'documenti non classificati', 'famiglia'] = 'altro'
df2021 = df2021[~df2021['nome_epc'].isna()]

df2021['original'] = df2021['nome_epc']
# Keep only alphabetic characters and spaces
df2021['nome_epc'] = df2021['original'].apply(lambda x: ''.join([c for c in x if c.isalpha() or c.isspace()]))
df2021['nome_epc'] = df2021['nome_epc'].str.replace(r'\s{2,}', ' ', regex=True)

# Trim leading and trailing whitespaces
df2021 = df2021.applymap(lambda x: x.strip() if isinstance(x, str) else x)

C:\Users\sophia.dellarciprete\AppData\Local\Temp\ipykernel_5764\2740716681.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df2021 = df2021.applymap(lambda x: x.lower() if isinstance(x, str) else x)
C:\Users\sophia.dellarciprete\AppData\Local\Temp\ipykernel_5764\2740716681.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df2021 = df2021.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [4]:
filtered_df = df2021[['famiglia', 'nome_epc', 'original']]
df_clean = filtered_df.drop_duplicates()

In [10]:
nltk.download('stopwords')
nlp = spacy.load('it_core_news_sm')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sophia.dellarciprete\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [11]:
def procesar_texto(cadena):
    sinpuntuacion = [c for c in cadena if c not in string.punctuation]
    sinpuntuacion =' '.join(sinpuntuacion)
    palabras = sinpuntuacion.split()
    cadena_limpia = [palabra for palabra in palabras if palabra not in stopwords.words("italian")]
    return cadena_limpia

transformador = CountVectorizer(analyzer=procesar_texto).fit(df_clean['nome_epc'])

In [14]:
final_dataframe= df_clean[df_clean["famiglia"] == 'altro'] # To use once the model is working
df_clean = df_clean[df_clean["famiglia"] != 'altro']

In [15]:
x_train, x_test, y_train, y_test = train_test_split(df_clean["nome_epc"], df_clean["famiglia"],test_size=0.3)

pipeline = Pipeline([
('vectorizar',CountVectorizer(analyzer = procesar_texto)),
('transformar', TfidfTransformer()), 
('clasificar', RandomForestClassifier())
])

In [16]:
pipeline.fit(x_train, y_train)

Pipeline(steps=[('vectorizar',
                 CountVectorizer(analyzer=<function procesar_texto at 0x00000194C42A3B00>)),
                ('transformar', TfidfTransformer()),
                ('clasificar', RandomForestClassifier())])

In [17]:
predicciones = pipeline.predict(x_test)

In [19]:
print(classification_report(y_test, predicciones)) 

c:\Users\sophia.dellarciprete\.conda\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sophia.dellarciprete\.conda\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                              precision    recall  f1-score   support

                             atti giudiziali       0.62      0.57      0.60      1624
                    certificazioni contabili       0.98      0.98      0.98      4342
                                     closing       0.00      0.00      0.00         3
                               comunicazioni       0.94      0.87      0.90      4834
         contratti e documentazione relativa       0.97      0.77      0.86       895
                                    delibere       0.00      0.00      0.00         1
                                     fatture       0.99      1.00      0.99     59838
          garanzie e documentazione relativa       0.96      0.83      0.89       959
                                     imposte       0.12      0.02      0.04       143
passaggi di status e relativa documentazione       0.60      0.96      0.74       479
                                     perizie       0.

c:\Users\sophia.dellarciprete\.conda\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
predictions = pipeline.predict(final_dataframe['nome_epc'])

In [27]:
final_dataframe['prev'] = predictions

In [29]:
final_dataframe = final_dataframe.drop(columns=['famiglia'])
final_dataframe.rename(columns={'prev': 'famiglia'}, inplace=True)
final_dataframe

,nome_epc,original,famiglia
1,inserimento acconto,inserimento acconto,comunicazioni
3,allega documento,allega documento,garanzie e documentazione relativa
4,documentogenerico,documentogenerico___,atti giudiziali
7,legfiimm,legfiimm_2890__,perizie
8,,_130502_145_221668,atti giudiziali
...,...,...,...
1298494,propostedidelibera,proposte_di_delibera_435864__0,proposte
1298600,quietanza f,quietanza f23,imposte
1298674,legnotule,legnotule_459056_1_,fatture
1298745,legnotule,legnotule_451793_1_,fatture


In [30]:
result_dataframe = pd.concat([final_dataframe, df_clean], ignore_index=True)

In [38]:
prova = df2021.merge(result_dataframe, on='original', how='left')
prova

,num_prat,famiglia_x,des_tipodoc,nomefile,nome_epc_x,url,original,nome_epc_y,famiglia_y
0,23.0,fatture,pubblicitaï¿½,02560563.pdf,cpnotule,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,cpnotule_387336_1_,cpnotule,fatture
1,532.0,altro,altro,00511949.pdf,inserimento acconto,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,inserimento acconto,inserimento acconto,comunicazioni
2,532.0,altro,altro,00511949.pdf,inserimento acconto,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,inserimento acconto,inserimento acconto,certificazioni contabili
3,532.0,altro,altro,00511949.pdf,inserimento acconto,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,inserimento acconto,inserimento acconto,comunicazioni
4,532.0,altro,altro,00511949.pdf,inserimento acconto,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,inserimento acconto,inserimento acconto,proposte
...,...,...,...,...,...,...,...,...,...
7445846,1375939.0,altro,altro,03595358.pdf,allega documento,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,allega documento,allega documento,comunicazioni
7445847,1375939.0,altro,altro,03595358.pdf,allega documento,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,allega documento,allega documento,contratti e documentazione relativa
7445848,1375939.0,altro,altro,03595358.pdf,allega documento,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,allega documento,allega documento,garanzie e documentazione relativa
7445849,1375939.0,altro,altro,03595358.pdf,allega documento,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,allega documento,allega documento,certificazioni contabili


In [40]:
prova.famiglia_y.unique()

array(['fatture', 'comunicazioni', 'certificazioni contabili', 'proposte',
       'garanzie e documentazione relativa', 'visure', 'atti giudiziali',
       'imposte', 'contratti e documentazione relativa',
       'passaggi di status e relativa documentazione', 'closing',
       'perizie', "pubblicita'", 'delibere'], dtype=object)

In [43]:
counts_dict = {}

# Iterate over unique 'famiglia' values
for famiglia_value in prova['famiglia_y'].unique():
    
    # Filter DataFrame for the current 'famiglia'
    filtered = prova[prova['famiglia_y'] == famiglia_value]
    
    # Count occurrences of each 'nome_epc' for the current 'famiglia'
    nome_epc_counts = filtered['nome_epc_y'].value_counts().to_dict()
    
    # Add counts to the dictionary
    counts_dict[famiglia_value] = nome_epc_counts

counts_df = pd.DataFrame(counts_dict)
counts_df

,fatture,comunicazioni,certificazioni contabili,proposte,garanzie e documentazione relativa,visure,atti giudiziali,imposte,contratti e documentazione relativa,passaggi di status e relativa documentazione,closing,perizie,pubblicita',delibere
cpnotule,171108.0,571.0,2.0,35.0,4.0,4.0,2441.0,304.0,1.0,NaN,1.0,82.0,232.0,NaN
documentogenerico,128920.0,128920.0,128920.0,128920.0,128920.0,128920.0,257840.0,128920.0,128920.0,128920.0,128920.0,128920.0,128920.0,128920.0
documento note sulla procedura,93298.0,93298.0,93298.0,93298.0,186596.0,93298.0,93298.0,93298.0,93298.0,93298.0,93298.0,93298.0,93298.0,NaN
legnotule,73161.0,22.0,NaN,12.0,1.0,NaN,84.0,94.0,NaN,NaN,NaN,3.0,NaN,NaN
il legale esterno ha esaminato comunicazione del professionista delegato di fissazione della vendita,20500.0,20500.0,NaN,20500.0,20500.0,NaN,41002.0,NaN,NaN,20500.0,NaN,20500.0,20502.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fattura aste annunci pagata dal delegato col fondo spese,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
sollecito versamento contributo pubblicitï,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
versamento contributo pubblicitï vendita,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
il legale esterno ha trasmesso al professionista la quietanza di pagamento della fattura,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN


In [44]:
max_family_for_row = counts_df.idxmax(axis=1)

# Create a new DataFrame with 'word' as index and 'famiglia' as the corresponding column
result_df = pd.DataFrame({'word': max_family_for_row.index, 'famiglia': max_family_for_row.values})

result_df

,word,famiglia
0,cpnotule,fatture
1,documentogenerico,atti giudiziali
2,documento note sulla procedura,garanzie e documentazione relativa
3,legnotule,fatture
4,il legale esterno ha esaminato comunicazione d...,atti giudiziali
...,...,...
22801,fattura aste annunci pagata dal delegato col f...,pubblicita'
22802,sollecito versamento contributo pubblicitï,pubblicita'
22803,versamento contributo pubblicitï vendita,pubblicita'
22804,il legale esterno ha trasmesso al professionis...,pubblicita'


In [47]:
done = df2021.merge(result_df, left_on='nome_epc', right_on='word', how='left')

In [54]:
done = done[done['famiglia_x']== 'altro']
done = done.drop(columns=['famiglia_x','word', 'nome_epc'] )
done.rename(columns={'famiglia_y': 'famiglia'}, inplace=True)
done.rename(columns={'original': 'nome_epc'}, inplace=True)
done

,num_prat,des_tipodoc,nomefile,url,nome_epc,famiglia
1,532.0,altro,00511949.pdf,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,inserimento acconto,comunicazioni
3,946.0,altro,00946317.pdf,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,allega documento,garanzie e documentazione relativa
4,1538.0,altro,02080862.pdf,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,documentogenerico___,atti giudiziali
6,1599.0,altro,00331905.pdf,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,allega documento,garanzie e documentazione relativa
7,1599.0,altro,00839638.pdf,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,legfiimm_2890__,perizie
...,...,...,...,...,...,...
1297928,1367400.0,altro,03384753.zip,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,allega documento,garanzie e documentazione relativa
1297958,1372272.0,altro,03460145.pdf,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,documentogenerico___,atti giudiziali
1297967,1372779.0,altro,03597572.pdf,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,ritiro copia/e esecutiva/e del titolo,fatture
1297971,1374833.0,altro,03484174.zip,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,allega documento,garanzie e documentazione relativa


In [59]:
done.to_csv('data/zeus_download_doc_digitalizzati20210514_exALTRO.csv', index=False)